In [1]:
import numpy as np
import openai
import pandas as pd
import pickle
import tiktoken

COMPLETIONS_MODEL = "text-davinci-003"
EMBEDDING_MODEL = "text-embedding-ada-002"

In [2]:
openai.api_key = ''#input your own openai api key

In [3]:
df= pd.read_csv('dataset_pu_dg_token_terbaru.csv')

In [4]:
df

,kategori,pasal,konten,tokens
0,PERATURAN MENTERI PEKERJAAN UMUM REPUBLIK INDO...,13/PRT/M/2011,TATA CARA PEMELIHARAAN DAN PENILIKAN JALAN,65
1,BAB I\nKETENTUAN UMUM,pasal 1,Dalam Peraturan Menteri ini yang dimaksud deng...,2355
2,"BAB II\nMAKSUD, TUJUAN, DAN LINGKUP PENGATURAN",pasal 2,(1) Pengaturan tata cara pemeliharaan dan peni...,413
3,BAB III\nRENCANA UMUM PEMELIHARAAN JALAN,Pasal 3,(1) Penyelenggara jalan wajib menyusun rencana...,129
4,BAB III\nRENCANA UMUM PEMELIHARAAN JALAN,Pasal 4,(1) Sistem informasi sebagaimana dimaksud dala...,146
5,BAB III\nRENCANA UMUM PEMELIHARAAN JALAN,Pasal 5,(1) Sistem manajemen aset sebagaimana dimaksud...,614
6,BAB III\nRENCANA UMUM PEMELIHARAAN JALAN,Pasal 6,(1) Rencana Penanganan pemeliharaan jalan seba...,1036


In [5]:
df = df.set_index(["kategori", "pasal"])
print(f"{len(df)} rows in the data.")

7 rows in the data.


In [6]:
df = df.loc[~df.index.duplicated(keep='first')]

In [7]:
def get_embedding(text: str, model: str=EMBEDDING_MODEL) -> list[float]:
    result = openai.Embedding.create(
      model=model,
      input=text
    )
    return result["data"][0]["embedding"]

In [8]:
def load_embeddings(fname: str) -> dict[tuple[str, str], list[float]]:
    """
    Read the document embeddings and their keys from a CSV.
    
    fname is the path to a CSV with exactly these named columns: 
        "title", "heading", "0", "1", ... up to the length of the embedding vectors.
    """
    
    df = pd.read_csv(fname, header=0)
    max_dim = max([int(c) for c in df.columns if c != "kategori" and c != "pasal"])
    return {
           (r.kategori, r.pasal): [r[str(i)] for i in range(max_dim + 1)] for _, r in df.iterrows()
    }

In [9]:
document_embeddings = load_embeddings("doc_pu_with_embeddings.csv")

In [10]:
def vector_similarity(x: list[float], y: list[float]) -> float:
    """
    Returns the similarity between two vectors.
    
    Because OpenAI Embeddings are normalized to length 1, the cosine similarity is the same as the dot product.
    """
    return np.dot(np.array(x), np.array(y))

def order_document_sections_by_query_similarity(query: str, contexts: dict[(str, str), np.array]) -> list[(float, (str, str))]:
    """
    Find the query embedding for the supplied query, and compare it against all of the pre-calculated document embeddings
    to find the most relevant sections. 
    
    Return the list of document sections, sorted by relevance in descending order.
    """
    query_embedding = get_embedding(query)
    
    document_similarities = sorted([
        (vector_similarity(query_embedding, doc_embedding), doc_index) for doc_index, doc_embedding in contexts.items()
    ], reverse=True)
    
    return document_similarities

In [11]:
MAX_SECTION_LEN = 3000
SEPARATOR = "\n* "
ENCODING = "cl100k_base"  # encoding for text-embedding-ada-002

encoding = tiktoken.get_encoding(ENCODING)
separator_len = len(encoding.encode(SEPARATOR))

f"Context separator contains {separator_len} tokens"

'Context separator contains 3 tokens'

In [12]:
def construct_prompt(question: str, context_embeddings: dict, df: pd.DataFrame) -> str:
    """
    Fetch relevant 
    """
    most_relevant_document_sections = order_document_sections_by_query_similarity(question, context_embeddings)
    
    chosen_sections = []
    chosen_sections_len = 0
    chosen_sections_indexes = []
     
    for _, section_index in most_relevant_document_sections:
        # Add contexts until we run out of space.        
        document_section = df.loc[section_index]
        
        chosen_sections_len += document_section.tokens + separator_len
        if chosen_sections_len > MAX_SECTION_LEN:
            break
            
        chosen_sections.append(SEPARATOR + document_section.konten.replace("\n", " "))
        chosen_sections_indexes.append(str(section_index))
            
    # Useful diagnostic information
    print(f"Selected {len(chosen_sections)} document sections:")
    print("\n".join(chosen_sections_indexes))
    
    header = """Jawab pertanyaan sejujur mungkin menggunakan konteks yang disediakan, dan jika jawabannya tidak terdapat dalam teks di bawah ini, katakan "Saya tidak tahu."\n\nContext:\n"""
    
    return header + "".join(chosen_sections) + "\n\n Q: " + question + "\n A:"

In [13]:
prompt = construct_prompt(
    "apa isi pasal 1",
    document_embeddings,
    df
)

print("===\n", prompt)

Selected 3 document sections:
('BAB I\nKETENTUAN UMUM ', 'pasal 1')
('BAB III\nRENCANA UMUM PEMELIHARAAN JALAN', 'Pasal 4')
('BAB III\nRENCANA UMUM PEMELIHARAAN JALAN', 'Pasal 3')
===
 Jawab pertanyaan sejujur mungkin menggunakan konteks yang disediakan, dan jika jawabannya tidak terdapat dalam teks di bawah ini, katakan "Saya tidak tahu."

Context:

* Dalam Peraturan Menteri ini yang dimaksud dengan: 1. Jalan adalah prasarana transportasi darat yang meliputi segala bagian jalan, termasuk bangunan pelengkap dan perlengkapannya yang diperuntukkan bagi lalulintas, yang berada pada permukaan tanah, di atas permukaan tanah, di bawah permukaan tanah dan/atau air, serta di atas permukaan air, kecuali jalan kereta api, jalan lori, dan jalan kabel. 2. Jalan Umum adalah jalan yang diperuntukan bagi lalu lintas umum. 3. Bangunan pelengkap adalah bangunan untuk mendukung fungsi dan keamanan konstruksi jalan yang meliputi jembatan, terowongan, ponton, lintas atas (flyover, elevated road), lintas b

In [20]:
COMPLETIONS_API_PARAMS = {
    # We use temperature of 0.0 because it gives the most predictable, factual answer.
    "temperature": 0.0,
    "max_tokens": 1000,
    "model": COMPLETIONS_MODEL,
}

In [21]:
def answer_query_with_context(
    query: str,
    df: pd.DataFrame,
    document_embeddings: dict[(str, str), np.array],
    show_prompt: bool = False
) -> str:
    prompt = construct_prompt(
        query,
        document_embeddings,
        df
    )
    
    if show_prompt:
        print(prompt)

    response = openai.Completion.create(
                prompt=prompt,
                **COMPLETIONS_API_PARAMS
            )

    return response["choices"][0]["text"].strip(" \n")

In [23]:
answer_query_with_context("PERATURAN MENTERI PEKERJAAN UMUM REPUBLIK INDONESIA  ?", df, document_embeddings)

Selected 2 document sections:
('BAB I\nKETENTUAN UMUM ', 'pasal 1')
('BAB III\nRENCANA UMUM PEMELIHARAAN JALAN', 'Pasal 3')


'Peraturan Menteri Pekerjaan Umum Republik Indonesia adalah peraturan yang mengatur tentang jalan, jalan umum, bangunan pelengkap, perlengkapan jalan, bagian-bagian jalan, ruang manfaat jalan, ruang milik jalan, ruang pengawasan jalan, penyelenggara jalan, pemeliharaan jalan, rehabilitasi jalan, rekonstruksi jalan, penilikan jalan, pemrograman, perkerasan berpenutup, pembentukan kembali permukaan, dan Menteri.'

In [24]:
answer_query_with_context("bunyi pasal 1?", df, document_embeddings)

Selected 3 document sections:
('BAB I\nKETENTUAN UMUM ', 'pasal 1')
('BAB III\nRENCANA UMUM PEMELIHARAAN JALAN', 'Pasal 4')
('BAB III\nRENCANA UMUM PEMELIHARAAN JALAN', 'Pasal 3')


'Dalam Peraturan Menteri ini yang dimaksud dengan: 1. Jalan adalah prasarana transportasi darat yang meliputi segala bagian jalan, termasuk bangunan pelengkap dan perlengkapannya yang diperuntukkan bagi lalulintas, yang berada pada permukaan tanah, di atas permukaan tanah, di bawah permukaan tanah dan/atau air, serta di atas permukaan air, kecuali jalan kereta api, jalan lori, dan jalan kabel. 2. Jalan Umum adalah jalan yang diperuntukan bagi lalu lintas umum. 3. Bangunan pelengkap adalah bangunan untuk mendukung fungsi dan keamanan konstruksi jalan yang meliputi jembatan, terowongan, ponton, lintas atas (flyover, elevated road), lintas bawah (underpass), tempat parkir, gorong-gorong, tembok penahan, dan saluran tepi jalan dibangun sesuai dengan persyaratan teknis. 4. Perlengkapan Jalan adalah sarana yang dimaksudkan untuk keselamatan, keamanan, ketertiban, dan kelancaran lalu-lintas serta kemudahan bagi pengguna jalan dalam berlalu-lintas yang meliputi marka jalan, rambu lalu-lintas, 

In [25]:
answer_query_with_context("rehabilitas jalan adalah?", df, document_embeddings)

Selected 3 document sections:
('BAB I\nKETENTUAN UMUM ', 'pasal 1')
('BAB III\nRENCANA UMUM PEMELIHARAAN JALAN', 'Pasal 3')
('PERATURAN MENTERI PEKERJAAN UMUM REPUBLIK INDONESIA\nNOMOR 13/PRT/M/2011', '13/PRT/M/2011')


'Rehabilitasi jalan adalah kegiatan penanganan pencegahan terjadinya kerusakan yang luas dan setiap kerusakan yang tidak diperhitungkan dalam desain, yang berakibat menurunnya kondisi kemantapan pada bagian/tempat tertentu dari suatu ruas jalan dengan kondisi rusak ringan, agar penurunan kondisi kemantapan tersebut dapat dikembalikan pada kondisi kemantapan sesuai dengan rencana.'